<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질-답, 모두의노코드 질-답+댓 각각을 요약해 저장하는 코드(로컬)

In [ ]:
!pip install datasets

import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt
from datasets import Dataset, load_dataset
from google.colab import drive
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel, PreTrainedTokenizerFast, BartForConditionalGeneration
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Mounted at /content/drive


In [ ]:
# Load Model and Tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
model_name = "alaggung/bart-r3f"

tokenizer = AutoTokenizer.from_pretrained("alaggung/bart-r3f")
model = BartForConditionalGeneration.from_pretrained("alaggung/bart-r3f")

inputs = tokenizer("[BOS]" + "[SEP]".join(dialogue) + "[EOS]", return_tensors="pt")
outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    num_beams=num_beams,
    length_penalty=length_penalty,
    max_length=max_length,
    use_cache=True,
)
summarization = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(summarization)
# 어제 김치찌개를 먹어서 한식 말고 돈가스를 먹기로 했다.

In [ ]:
#카카오톡 질답세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/judge_answer_result_KcBERT_short_questiontrained.csv'

df1 = pd.read_csv(file_path, index_col = 0)

#커뮤니티 데이터와 결합하기 위해 url 열 추가
df1['url'] = [''] * len(df1)

# dataset = load_dataset("csv", data_files=file_path, column_names=["question", "answer", "questioner", "respondent", "date", "url"])

print(df1)

In [ ]:
#모두의노코드 질-답+댓 세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'

df2 = pd.read_excel(file_path, index_col = 0)

#카카오톡 데이터와 결합하기 위해 답변 작성자 열 추가
df2['respondent'] = [''] * len(df2)

#Slug가 존재하는 경우만 질문 원글로 판별
df2 = df2[pd.isna(df2['Slug']) == False]
#답변이 달리지 않은 질문글은 제외, 추후 댓글까지 활용 예정
df2 = df2[pd.isna(df2['답변']) == False]

#필요없는 열 삭제
df2 = df2.drop(['댓글', '관련 링크', '제목', '첨부 파일', 'Slug'], axis = 1)

#카카오톡 데이터와 결합하기 위해 열 이름 변경
df2 = df2.rename(columns = {'내용' : 'question', '답변' : 'answer', '작성일' : 'date', '작성자' : 'questioner', '게시글 링크' : 'url'})

# dataset = load_dataset("csv", data_files = file_path, column_names = ["question", "answer", "questioner", "respondent", "date", "url"])

print(df2)

In [ ]:
print(type(df1.iloc[2]['date']))

<class 'str'>


In [ ]:
# df2['date'] = df2['date'].astype(str)

In [ ]:
print(type(df2.iloc[3]['date']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
# #두 데이터프레임 합치고 Dataset으로 변환하기
df = pd.concat([df1, df2], ignore_index = True)
print(df)
dataset = Dataset.from_pandas(df)

print(dataset)

In [ ]:
# Ensure the classifier is using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
# # 데이터셋 전처리 (예제)
# texts = ["This is a long text that needs to be summarized.", "Here's another text for summarization."]
# tokenized_inputs = tokenizer(texts, padding="longest", truncation=True, return_tensors="pt", max_length=1024)

# print(tokenized_inputs)

{'input_ids': tensor([[    0, 15054,   303, 15868,  1700, 15868, 22911, 25674, 24687, 18090,
         23795,   315, 19133, 16881, 21514,   300, 17752,   314, 27141, 19306,
           300, 18482,   316, 20461, 15562, 26147, 17752,   245,     1],
        [    0, 16391, 14879,   300,   238,   314,  1700, 15195,   310, 16160,
         14879, 18090, 23795,   315, 21579, 18482,   316, 20461, 15562, 26147,
         19334,   245,     1,     3,     3,     3,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0]])}


In [ ]:
# 데이터셋 질문 전처리 함수, Dataset 열을 인자로 받는다.
def preprocess_question(examples):
    return tokenizer(examples['question'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#토큰화된 질문 길이 통계량 확인
# Encoding question
tokenized_question_inputs = tokenizer(df['question'].to_list())

# 토큰화된 질문에서 각 질문의 길이 계산
lengths = [len(input_id) for input_id in tokenized_question_inputs['input_ids']]

# 최대 길이를 갖는 질문의 인덱스 찾기
max_length_index = [index for index, length in enumerate(lengths) if length == 972]

# 해당 인덱스의 원본 질문 출력
for index in max_length_index:
    print(df['question'].iloc[index])

print('토큰화된 질문의 최대 길이 :', max(len(sample) for sample in tokenized_question_inputs['input_ids']))
print('토큰화된 질문의 평균 길이 :', sum(map(len, tokenized_question_inputs['input_ids']))/len(tokenized_question_inputs['input_ids']))

plt.hist([len(sample) for sample in tokenized_question_inputs['input_ids']], bins = 100)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
print(len(tokenizer('버블 softr 웹플로우 아임웹 글라이드는 언급되는데 워드프레스는 노코드에 언급이 없어서 버블과 워드프레스는')['input_ids']))

39


In [ ]:
#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

In [ ]:
#모델로 question summary 리스트 생성
question_summary = []

# Encoding question
question_text_dict = {"question" : dataset["question"][1:]} #dataset["question"]의 첫 요소인 'question'삭제
question_text = Dataset.from_dict(question_text_dict)
tokenized_question_inputs = question_text.map(preprocess_question, num_proc = 4, batched = True, batch_size = 128)

#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

loader = DataLoader(tokenized_question_inputs, batch_size=16, collate_fn=collate_fn)

# Question summary ids 생성,배치 단위
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  question_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 100,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )
  #생성한 Question summary ids를 다시 tokenizer가 있는 cpu로 이동
  question_summary_ids = question_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  question_summary.append(question_summary_ids)

# Decoding question summary (list)
qsum = []

#배치 단위로 decode 처리
for question_summary_result in question_summary:
  qsum_batch = [tokenizer.decode(question_summary_result[i], skip_special_tokens = True) for i in range(len(question_summary_result))]
  qsum.append(qsum_batch)

#배치 풀기
qsum = sum(qsum, [])
df_qsum = pd.DataFrame(qsum)

print(df_qsum)

In [ ]:
# 데이터셋 답변 전처리 함수, Dataset 열을 인자로 받는다. 질문 전처리 함수와 공통적으로 사용하지 않은 이유는 examples['answer']형태로 입력하지 않으면 오류가 발생한다.
def preprocess_answer(examples):
    return tokenizer(examples['answer'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#토큰화된 답변 길이 통계량 확인
# Encoding answer
tokenized_answer_inputs = tokenizer(df['answer'].to_list())

print('토큰화된 답변의 최대 길이 :', max(len(sample) for sample in tokenized_answer_inputs['input_ids']))
print('토큰화된 답변의 평균 길이 :', sum(map(len, tokenized_answer_inputs['input_ids']))/len(tokenized_answer_inputs['input_ids']))

plt.hist([len(sample) for sample in tokenized_answer_inputs['input_ids']], bins = 100)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
#모델로 answer summary 리스트 생성
answer_summary = []

# Encoding answer
answer_text_dict = {"answer" : dataset["answer"][1:]} #dataset["answer"]의 첫 요소인 'answer'삭제
answer_text = Dataset.from_dict(answer_text_dict)
tokenized_answer_inputs = answer_text.map(preprocess_answer, num_proc = 4, batched = True, batch_size = 128)

loader = DataLoader(tokenized_answer_inputs, batch_size=16, collate_fn=collate_fn)

# Answer summary ids 생성,배치 단위, question때와 다른 하이퍼파라미터 필요(요약길이)
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  answer_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 200,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )

  #생성한 Answer summary ids를 다시 tokenizer가 있는 cpu로 이동
  answer_summary_ids = answer_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  answer_summary.append(answer_summary_ids)

# Decoding question summary (list)
asum = []

#배치 단위로 decode 처리
for answer_summary_result in answer_summary:
  asum_batch = [tokenizer.decode(answer_summary_result[i], skip_special_tokens = True) for i in range(len(answer_summary_result))]
  asum.append(asum_batch)

#배치 풀기
asum = sum(asum, [])
df_asum = pd.DataFrame(asum)

print(df_asum)

In [ ]:
#데이터셋에 새로운 열 추가
print(len(dataset))

dataset_dict = {'question' : dataset['question'][1:], 'answer' : dataset['answer'][1:], 'question_summary' : qsum, 'answer_summary' : asum, 'question_withurl' : dataset['question_withurl'][1:], 'answer_withurl' : dataset['answer_withurl'][1:], 'questioner' : dataset['questioner'][1:], 'respondent' : dataset['respondent'][1:], 'date' : dataset['date'][1:], 'url' : dataset['url'][1:]}
dataset = Dataset.from_dict(dataset_dict)

print(dataset)
# dataset.add_column("qsum", qsum)
# dataset.add_column("asum", asum)

772
Dataset({
    features: ['question', 'answer', 'question_summary', 'answer_summary', 'question_withurl', 'answer_withurl', 'questioner', 'respondent', 'date', 'url'],
    num_rows: 771
})


In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/summary_result_short.csv'

dataset.to_csv(save_path)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1882029